In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

from bama_class import Bama

# Read Data from DataBase

In [2]:
# read data from database
bama = Bama()
data_df = bama.read_data_from_db()


# clean up the data

## remove extra columns

In [3]:
# remove extra columns
columns = ['detail.code', 
           'detail.title',
           'detail.subtitle', 
           'detail.trim', 
           'detail.year',
           'detail.mileage', 
           'detail.location', 
           'detail.specialcase',
           'detail.transmission', 
           'detail.fuel', 
           'detail.color',
           'detail.body_color', 
           'detail.inside_color', 
           'detail.body_status',
           'detail.body_type',
           'detail.body_type_fa', 
           'detail.cylinder_fa', 
           'specs.volume', 
           'specs.engine', 
           'specs.acceleration', 
           'specs.fuel',
           'price.type',
           'price.price']

data_df = data_df[columns].copy()

## get data info and remove rows with missing values

In [4]:
# get data info
data_df.info()

In [5]:
# remove columns with missing values
data_df.drop(columns=['detail.specialcase'], inplace=True)
data_df.dropna(inplace=True)


## delete columns with duplicate values

In [6]:
deleted_columns = ['detail.subtitle',
                   'detail.color',
                   'detail.body_type_fa',
                   'detail.code'
]

data_df.drop(columns=deleted_columns, inplace=True)

## remove duplicate rows

In [7]:
# remove duplicate rows
data_df.drop_duplicates(inplace=True)

## convert hijri date to gregorian date

In [8]:
# change data types
data_df['detail.year'] = data_df['detail.year'].astype(int)

# convert hijri year to gregorian year
hirji_year = data_df[data_df['detail.year'] < 1500]
data_df.loc[hirji_year.index, 'detail.year'] = hirji_year['detail.year'] + (2024 - 1403)
        

## convert detail.mileage to integer

In [9]:
# convert detail.mileage to integer
data_df.loc[data_df['detail.mileage'].str.contains('صفر کیلومتر'), 'detail.mileage'] = '0'

# remove 'km' from detail.mileage
data_df['detail.mileage'] = data_df['detail.mileage'].str.replace('km', '').str.strip()
# remove ',' from detail.mileage
data_df['detail.mileage'] = data_df['detail.mileage'].str.replace(',', '').str.strip()
# remove 'کارکرده' from detail.mileage
data_df['detail.mileage'] = data_df['detail.mileage'].str.replace('کارکرده', '-1').str.strip()

# convert detail.mileage to integer
data_df['detail.mileage'] = data_df['detail.mileage'].astype(int)

# change -1 to NaN
data_df.loc[data_df['detail.mileage'] == -1, 'detail.mileage'] = np.nan

# replace nan values with mean
data_df.fillna({'detail.mileage': data_df['detail.mileage'].mean()}, inplace=True)

## convert detail.cylinder_fa to integer

In [10]:
# remove سیلندر from detail.cylinder_fa
data_df['detail.cylinder_fa'] = data_df['detail.cylinder_fa'].str.replace('سیلندر', '').str.strip()

# convert detail.cylinder_fa to integer
data_df['detail.cylinder_fa'] = data_df['detail.cylinder_fa'].astype(int)

## convert specs.volume to float

In [11]:
# convert specs.volume to float
# remove لیتر from specs.volume
data_df['specs.volume'] = data_df['specs.volume'].str.replace('لیتر', '').str.strip()

# convert specs.volume to float
data_df['specs.volume'] = data_df['specs.volume'].astype(float)

## convert specs.acceleration to float

In [12]:
# ## convert specs.acceleration to float
# remove ثانیه from specs.acceleration
data_df['specs.acceleration'] = data_df['specs.acceleration'].str.replace('ثانیه', '').str.strip()

# replace '/' with '.' in specs.acceleration
data_df['specs.acceleration'] = data_df['specs.acceleration'].str.replace('/', '.').str.strip()

# convert specs.acceleration to float
data_df['specs.acceleration'] = data_df['specs.acceleration'].astype(float)

## convert specs.fuel to float

In [13]:
## convert specs.fuel to float
# remove لیتر در صد کیلومتر from specs.fuel
data_df['specs.fuel'] = data_df['specs.fuel'].str.replace('لیتر در صد کیلومتر', '').str.strip()

# convert specs.fuel to float
data_df['specs.fuel'] = data_df['specs.fuel'].astype(float)

## remove price.type which is negotiable

In [14]:
## remove price.type which is negotiable
data_df = data_df[data_df['price.type'] != 'negotiable'].copy()
data_df

# remove , from price.price
data_df['price.price'] = data_df['price.price'].str.replace(',', '').str.strip()

# convert price.price to float
data_df['price.price'] = data_df['price.price'].astype(float)

# remove extra columns and change data types

In [15]:
data_df.drop(columns=['price.type'], inplace=True)

data_df = data_df.astype({'detail.title': 'str', 
                          'detail.trim': 'str',
                          'detail.year': 'int',
                          'detail.mileage': 'float',
                          'detail.location': 'str',
                          'detail.transmission': 'str',
                          'detail.fuel': 'str',
                          'detail.body_color': 'str',
                          'detail.inside_color': 'str',
                          'detail.body_status': 'str',
                          'detail.body_type': 'str',
                          'detail.cylinder_fa': 'int',
                          'specs.volume': 'float',
                          'specs.engine': 'str',
                          'specs.acceleration': 'float',
                          'specs.fuel': 'float',
                          'price.price': 'float'
                            })

data_df.info()


In [16]:
data_df

# Data Preprocessing

In [17]:
# get list of string columns
string_columns = data_df.select_dtypes(include='object').columns

# convert string columns to category with sklearn

label_encoder = LabelEncoder()

for column in string_columns:
    data_df[column] = label_encoder.fit_transform(data_df[column])

data_df

In [18]:
data_df.describe()

In [19]:
# split data to train, validation and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=42)
train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=42)

print(f'train_df: {train_df.shape}')
print(f'valid_df: {valid_df.shape}')
print(f'test_df: {test_df.shape}')

In [20]:
# set features and target
features = train_df.drop(columns=['price.price'])
target = train_df['price.price']

valid_features = valid_df.drop(columns=['price.price'])
valid_target = valid_df['price.price']

test_features = test_df.drop(columns=['price.price'])
test_target = test_df['price.price']


# use random forest to predict the price

In [21]:
# use random forest to predict price
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# create model with validation data
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)

model.fit(features, target)

# predict validation data
valid_predictions = model.predict(valid_features)

# calculate mean squared error
train_mse = mean_squared_error(target, model.predict(features))
valid_mse = mean_squared_error(valid_target, valid_predictions)

print(f'train mse: {train_mse}')
print(f'valid mse: {valid_mse}')


In [22]:
# predict test data
test_predictions = model.predict(test_features)

# calculate mean squared error
test_mse = mean_squared_error(test_target, test_predictions)

print(f'test mse: {test_mse}')

In [23]:
# plot predictions vs actual
plt.figure(figsize=(10, 6))
plt.scatter(test_target, test_predictions, alpha=0.6)

plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.title('Actual Price vs Predicted Price')

# plot line
x = np.linspace(0, 7*10**10, 100)
y = x
plt.plot(x, y, color='r')


plt.show()
